In [20]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.document_loaders import WebBaseLoader
import bs4

In [21]:
loader = WebBaseLoader(
    web_paths= ("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    )
)

In [22]:
docs = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

splits = text_splitter.split_documents(docs) # text_splitter < 다시저장

In [24]:
pip install rank_bm25

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


vectorstore = Chroma.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={"k" : 1, "fetch_k" : 4}
)
# 앙상블로 한거구나

In [30]:
bm25_retriever = BM25Retriever.from_documents(splits)

bm25_retriever.k = 2

In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever], # 앙상블 써서 각각의 결과값에 가중치를 줘서 사용
                weights=[0.2, 0.8])

In [34]:
docs = ensemble_retriever.invoke("삼일회계법인,pwc")

In [35]:
docs

[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='지난 2년 동안 국내 시가총액 상위 100대 기업의 주주환원 규모가 35% 이상 증가한 것으로 나타났다. 특히 KT&G는 시가총액 대비 주주환원 비율에서 1위를 기록했다. 6일 기업데이터연구소 CEO스코어에 따르면 \n\n\n한경비즈니스\n\n25분전'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='문화일보\n\n25분전\n\n\n\n\n\n\n\n\n2년 새 35% 늘어난 주주환원...KT&G, 시총 대비 환원율 1위')]

In [37]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [38]:
template = """
당신은 AI 언어 모델 조수입니다. 당신의 임무는 주어진 사용자 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 다섯 가지 다른 버전을 생성하는 것입니다.
사용자 질문에 대한 여러 관점을 생성함으로써, 거리 기반 유사성 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다.
각 질문은 새 줄로 구분하여 제공하세요. 원본 질문: {question}
"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


In [39]:
generate_queries = (
    prompt_perspectives
    | ChatOpenAI(model_name = "chatgpt-4o-latest", temperature=0)
    | StrOutputParser()
    | (lambda x : x.split("\n"))
)

In [41]:
from langchain.load import dumps, loads

In [ ]:
def reciprocal_rank_fusion(results, k=60, top_n=2):
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(docs)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)
   
    reranked_results = [ (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x : x[1], reverse=True)
    ]
    return reranked_results[:top_n]




chain = generate_queries | ensemble_retriever.map() | reciprocal_rank_fusion

In [48]:
from langchain_core.runnables import RunnablePassthrough


template = """다음 맥락을 바탕으로 질문에 답변할 것
{context}


질문: {question}
"""


prompt = ChatPromptTemplate.from_template(template)

In [49]:
final_chain = (
    {
        "context" : chain,
        "question" : RunnablePassthrough()
    }
    | prompt | ChatOpenAI(model_name="chatgpt-4o-latest", temperature=0)
    | StrOutputParser()


)


rt = final_chain.invoke("오늘의 증시")


from openai import OpenAI
client = OpenAI()
response = client.audio.speech.create(
    model="tts-1",
    voice="onyx",
    input=rt
    )


response.stream_to_file("./output.mp3")

/tmp/ipykernel_4765/1710383296.py:11: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  reranked_results = [ (loads(doc), score)
/tmp/ipykernel_4765/2910983370.py:25: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("./output.mp3")


In [26]:
from langchain_text_splitters import CharacterTextSplitter
from unstructured.partition.pdf import partition_pdf

/home/godldy73/miniconda3/envs/openai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def extract_pdf_elements(path, fname):
    return partition_pdf(
        filename=path + fname,
        extract_images_in_pdf=True,  # PDF에서 이미지를 추출
        infer_table_structure=True,  # 테이블 구조를 추론
        chunking_strategy="by_title",  # 타이틀을 기준으로 텍스트를 블록으로 분할
        max_characters=4000,  # 최대 4000자로 텍스트 블록을 제한
        new_after_n_chars=3800,  # 3800자 이후에 새로운 블록 생성
        combine_text_under_n_chars=2000,  # 2000자 이하의 텍스트는 결합
        image_output_dir_path=path,  # 이미지가 저장될 경로 설정
        # image_output_dir_path=os.path.join(os.getcwd(),"figures"),
    )

In [ ]:
raw_data = extract_pdf_elements("../data/", "최민석_자기소개서_.pdf")